In [95]:
import gspread

In [96]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Integer, String, Float, DateTime, func
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, sessionmaker
from typing import Optional

In [97]:
gc = gspread.service_account(filename='credentials.json')

In [98]:
SHEET_ID = r'1WLH9Qes7OJeIyNJ7RLmKz9dJ_6XtGvbSQQOsprAfV54'
ws = gc.open_by_key(SHEET_ID).worksheet('Ventas')

In [99]:
def extraer_datos(ws):    
    """
    Función para extraer los datos de la hoja de Google Sheets y convertirlos en DataFrame.
    """

    print("Extracción de datos")
    
    values = np.array(ws.get('A1:F5'))
    df = pd.DataFrame(data=values[1:], columns=values[0])
    return df


def transformar_datos(df: pd.DataFrame, fmt='%Y-%m-%d'):
    """
    Función para declarar el tipo de datos de cada variable y el cálculo de total de ventas.
    """

    print("Transformación de datos")
    
    if 'Fecha' in df.columns:
        try:
            df['Fecha'] = pd.to_datetime(df['Fecha'], format=fmt)
        except ValueError as e:
            print(f"Problema con el formato convirtiendo 'Fecha' con '{fmt}': {e}")

            df['Fecha'] = pd.to_datetime(
                df['Fecha'],
                errors='coerce',
                infer_datetime_format=True,
                dayfirst=False
            )
        except Exception as e:
            print(f"[{type(e).__name__}] {e}")

        nas = df.isna().T.sum(axis=1)
        print("Nulos en la base de datos", end='\n\n')
        print(nas)

        dtypes = {
            'ID': int,
            'Producto': str,
            'Cantidad': int,
            'Precio': float,
            'Cliente_ID': int
        }

        for x, y in dtypes.items():
            df[x] = df[x].astype(y)

        df['Total'] = df['Cantidad'] * df['Precio'] # Cálculo nuevo
        
        return df


class Base(DeclarativeBase):
    pass


class Venta(Base):
    __tablename__ = 'Ventas'
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    producto: Mapped[str] = mapped_column(String(100))
    cantidad: Mapped[int] = mapped_column(Integer)
    precio: Mapped[float] = mapped_column(Float)
    fecha: Mapped[DateTime] = mapped_column(DateTime)
    cliente_id: Mapped[int] = mapped_column(Integer)
    total: Mapped[Optional[float]] = mapped_column(Float, nullable=True)

In [100]:
def main():
    ventas = extraer_datos(ws)
    ventas = transformar_datos(ventas)
    
    display(ventas)
    
    # Conexión y Creación de BD con SQLAlchemy
    engine = create_engine('sqlite:///ventas.db', echo=True) # echo para logging
    Base.metadata.create_all(engine) # Crea tablas si no existen
    
    # Sesión Factory
    Session = sessionmaker(bind=engine)
    
    # Carga (Load)
    
    array_ventas = []
    for _, row in ventas.iterrows():
        venta = Venta(
            id=int(row['ID']),
            producto=row['Producto'],
            cantidad=int(row['Cantidad']),
            precio=row['Precio'],
            fecha=row['Fecha'],
            cliente_id=int(row['Cliente_ID']),
            total=row['Total']
        )    
        array_ventas.append(venta)
    
    with Session() as session:
        try:
            session.add_all(array_ventas)
            session.commit()
            print("Datos cargados exitosamente en ventas.db.")
        except Exception as e:
            session.rollback()
            print(f"Error en carga: {e}")
    
    # Ejecución de Consultas con ORM
    with Session() as session:
        
        # Consulta 1: Todas las ventas
        todas_ventas = session.query(Venta).all()
        print("\nTodas las ventas.")
        for v in todas_ventas:
            print(f"ID: {v.id}, Producto: {v.producto}, Total: {v.total}")
            
        # Consulta 2: Total de ventas después de una fecha (filtro y agregación)
        total_ventas = session.query(func.sum(Venta.total)).filter(Venta.fecha > pd.to_datetime("2025-03-01")).scalar()
        print(f"\nTotal de ventas después de 2025-03-01: {total_ventas}")
        
        # Consulta 3: Ventas por cliente (group by implicito en query)
        ventas_por_cliente = session.query(Venta.cliente_id, func.count(Venta.id)).group_by(Venta.cliente_id).all()
            
        print("\nVentas por cliente:")
        for cliente, count in ventas_por_cliente:
            print(f"Cliente ID: {cliente}, Número de ventas: {count}")

In [101]:
if __name__ == '__main__':
    main()

Extracción de datos
Transformación de datos
Nulos en la base de datos

ID            0
Producto      0
Cantidad      0
Precio        0
Fecha         0
Cliente_ID    0
dtype: int64


,ID,Producto,Cantidad,Precio,Fecha,Cliente_ID,Total
0,1,Libro,2,15.5,2025-01-10,101,31.0
1,2,Laptop,1,800.0,2025-02-15,102,800.0
2,3,Teléfono,3,300.0,2025-03-20,101,900.0
3,4,Silla,4,50.0,2025-04-25,103,200.0


2025-09-16 19:31:44,905 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-16 19:31:44,906 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Ventas")
2025-09-16 19:31:44,906 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-16 19:31:44,907 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Ventas")
2025-09-16 19:31:44,907 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-16 19:31:44,908 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Ventas" (
	id INTEGER NOT NULL, 
	producto VARCHAR(100) NOT NULL, 
	cantidad INTEGER NOT NULL, 
	precio FLOAT NOT NULL, 
	fecha DATETIME NOT NULL, 
	cliente_id INTEGER NOT NULL, 
	total FLOAT, 
	PRIMARY KEY (id)
)


2025-09-16 19:31:44,909 INFO sqlalchemy.engine.Engine [no key 0.00034s] ()
2025-09-16 19:31:44,916 INFO sqlalchemy.engine.Engine COMMIT
2025-09-16 19:31:44,918 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-16 19:31:44,920 INFO sqlalchemy.engine.Engine INSERT INTO "Ventas" (id, producto, cantidad, precio, fecha, cliente_id, to